In [1]:
###

# Deep Learning (RNN) Demo for Load Forecasting

### Step 1: Import all the packages needed

In [2]:
import tensorflow as tf
from tensorflow.models.rnn import rnn, rnn_cell
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib as mp
import random as rd
import argparse
import os, sys
import csv
import math
import time
import matplotlib.pyplot as pl

### Step 2: setting all global parameters -- sec 2 network configuration

In [3]:
time1 = time.time() # set up counter to record run time
data_dir = './data/' # directory contains input data
num_epoches = 1500 # training epoches for each customer samples
n_steps = 48 # input size
cus_num = 10
test_batch_size = 70*48*cus_num # days of a batch
test_minibatch_size = 70*48
train_batch_size = 10*48*cus_num
feature_size = 1 # same time of a week
n_hidden = 20 # input size
num_layers = 5
n_output = 1
Rs = 10
cus_count_list = []

### Step 4: define data generating function code. 
which generate a batch of batch-size large sequence data. the data is feature_size dims width and is a time series of float32 of steps steps. inputs and outputs are:

inputs:
----n_batch: number of samples in a batch
----steps: the sequence length of a sample data
----feature_size: dimensions of a single time step data frame

outputs:
----X inputs, shape(n_batch,steps,feature_size)
----Y outputs should be, shape(n_batch,)

In [4]:
def train_data_gen(totaltraindays,x_data,y_data,steps = 48, n_batch = train_batch_size):
    X = np.zeros((n_batch,steps,feature_size))
    Y = np.zeros((n_batch,feature_size))
    rang = range(totaltraindays) # test day sample range
    train_days_list = rd.sample(rang,n_batch) # pick unduplicated n indexes as examples
    #print totaltraindays
    tmpX = [x_data[i,0-steps:] for i in train_days_list]
    tmpY = [y_data[i,:] for i in train_days_list]
    X = np.array(tmpX).reshape(n_batch,steps,feature_size)
    Y = np.array(tmpY).reshape(n_batch,feature_size)
        
    return (X,Y)

In [5]:
def test_data_gen(x_data,y_data,steps = 48, n_batch = test_batch_size):
    X = np.zeros((n_batch,steps,feature_size))
    Y = np.zeros((n_batch,feature_size))
    #print x_data[:,0-steps:].shape,y_data.shape
    #print n_batch, steps
    X = x_data[:,0-steps:].reshape(n_batch,steps,feature_size)
    Y = y_data.reshape(n_batch,feature_size)
    return (X,Y)

### Step 5: construct RNN model

In [6]:
# create placeholder for x and y
x = tf.placeholder("float",[None,n_steps,feature_size])
istate = tf.placeholder("float",[None,num_layers*2*n_hidden])
y = tf.placeholder("float",[None,n_output])


# Define weights
weights = {
    'hidden': tf.Variable(tf.random_normal([feature_size, n_hidden])), # Hidden layer weights
    'out': tf.Variable(tf.random_normal([n_hidden, n_output]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_output]))
}

In [7]:
def RNN(_X, _istate, _weights, _biases):

    # input shape: (batch_size, n_steps, n_input)
    _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
    #_Z = tf.transpose(_Z,[1,0,2])
    # Reshape to prepare input to hidden activation
    _X = tf.reshape(_X, [-1, feature_size]) # (n_steps*batch_size, n_input)
    #_Z = tf.reshape(_Z, [-1, cus_num])
    # Linear activation
    _X = tf.matmul(_X, _weights['hidden']) + _biases['hidden']
    #_Z = tf.matmul(_Z, _weights['inp']) + _biases['inp']
    #_Q = tf.add(_X,_Z)
    # Define a lstm cell with tensorflow
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
    stacked_lstm_cell = rnn_cell.MultiRNNCell([lstm_cell]*num_layers)
    
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(0, n_steps, _X) # n_steps * (batch_size, n_hidden)

    # Get lstm cell output
    outputs, states = rnn.rnn(stacked_lstm_cell, _X, initial_state=_istate)

    # Linear activation
    # Get inner loop last output
    return tf.matmul(outputs[-1], _weights['out']) + _biases['out']

In [8]:
pred = RNN(x, istate, weights, biases)

#cost function 
cost = tf.reduce_mean(tf.pow(pred-y,2)) # cost function of this batch of data
cost2 = tf.abs(pred-y) # 
#compute parameter updates
#train_op = tf.train.GradientDescentOptimizer(0.008).minimize(cost)
optimizer = tf.train.RMSPropOptimizer(0.005, 0.3).minimize(cost)
optimizer2 = tf.train.RMSPropOptimizer(0.005, 0.3).minimize(cost2)

In [9]:
## iterating among all customers to find current training customer
result_final = []
#cus_id_forselect[cus_num]
sim_id_forecast = [0,266,295,431,465,597,615,627,736,798]
dis_id_forecast = [0,230,460,487,520,655,754,767,818,907]

for i in range(0,cus_num):
    #ii = cus_id[i]
    ii = i#sim_id_forecast[i]
    test_x_name = data_dir + 'test_x_' + str(ii) + '.csv'
    test_y_name = data_dir + 'test_y_' + str(ii) + '.csv'
    train_x_name = data_dir + 'train_x_' + str(ii) + '.csv'
    train_y_name = data_dir + 'train_y_' + str(ii) + '.csv'
    leng_list = []
    tmp_data = np.array(pd.read_csv(test_x_name,header = None))
    if i == 0:
        test_x_data = tmp_data[:,1:]
    else:
        test_x_data = np.concatenate((test_x_data,tmp_data[:,1:]),axis=0)
    
    # print test_x_data.dtype  data are stored as float64 double precision format
    tmp_data = np.array(pd.read_csv(test_y_name,header = None))
    if i == 0:
        test_y_data = tmp_data[:,1:]
    else:
        test_y_data = np.concatenate((test_y_data,tmp_data[:,1:]),axis=0)
    
    tmp_data = np.array(pd.read_csv(train_x_name,header = None))
    if i == 0:
        train_x_data = tmp_data[:,1:]
    else:
        train_x_data = np.concatenate((train_x_data,tmp_data[:,1:]),axis=0)
        
    tmp_data = np.array(pd.read_csv(train_y_name,header = None))
    if i == 0:
        train_y_data = tmp_data[:,1:]
    else:
        train_y_data = np.concatenate((train_y_data,tmp_data[:,1:]),axis=0)
    #print tmp_data[:,1:].shape[0]
    #leng_list.append(tmp_data[:,1:].shape[0])
traindays = train_y_data.shape[0]

In [10]:
R = np.zeros((Rs,cus_num))
for i in range(0,1):
    # generate test data
    test_x,test_y = test_data_gen(test_x_data,test_y_data,n_steps,test_batch_size)
    test_x = test_x.reshape(test_batch_size,n_steps,feature_size)
    ### Execute
    # Initializing the variables
    init = tf.initialize_all_variables()
    outp = []
    outlist = np.zeros([Rs,test_batch_size])
    with tf.Session() as sess:
        # Create a summary to monitor cost function
        #tf.scalar_summary("loss", cost)
        #tf.scalar_summary("loss2",cost2)
        # Merge all summaries to a single operator
        #merged_summary_op = tf.merge_all_summaries()

        # tensorboard info.# Set logs writer into folder /tmp/tensorflow_logs
        #summary_writer = tf.train.SummaryWriter('/tmp/tensorflow_logs', graph_def=sess.graph_def)

        #initialize all variables in the model
        sess.run(init)
        for k in range(num_epoches):
            #Generate Data for each epoch
            #What this does is it creates a list of of elements of length seq_len, each of size [batch_size,input_size]
            #this is required to feed data into rnn.rnn
            #print traindays
            X,Y = train_data_gen(traindays,train_x_data,train_y_data,n_steps,train_batch_size)
            X = X.reshape(train_batch_size,n_steps,feature_size)


            #Create the dictionary of inputs to feed into sess.run
            if k < 100:
                sess.run(optimizer2,feed_dict={x:X,y:Y,istate:np.zeros((train_batch_size,num_layers*2*n_hidden))})
            else:
                sess.run(optimizer,feed_dict={x:X,y:Y,istate:np.zeros((train_batch_size,num_layers*2*n_hidden))})   
            #perform an update on the parameters

            #loss1 = sess.run(cost, feed_dict = {x:X,y:Y,istate:np.zeros((train_batch_size,num_layers*2*n_hidden))} )
            #loss2 = sess.run(cost, feed_dict = {x:test_x,y:test_y,istate:np.zeros((test_batch_size,num_layers*2*n_hidden))} )            #compute the cost on the validation set
            #output_tmp = sess.run(pred,feed_dict = {x:X,y:Y,istate:np.zeros((train_batch_size,num_layers*2*n_hidden))} )
            #outp_train = output_tmp
            if k >= num_epoches-Rs:
                output_tmp = sess.run(pred,feed_dict = {x:test_x,y:test_y,istate:np.zeros((test_batch_size,num_layers*2*n_hidden))} )
                outp_test = output_tmp
            if k >= num_epoches-Rs:
                outlist[k-num_epoches+Rs,:] = outp_test.copy().T

            # Write logs at every iteration
            #summary_str = sess.run(merged_summary_op, feed_dict={x:test_x,y:test_y,istate:np.zeros((test_batch_size,num_layers*2*n_hidden))} )
            #summary_writer.add_summary(summary_str, k)
            #print "Iter " + str(k) + ", Minibatch Loss ---- Train = " + "{:.6f}".format(loss1)# + "; Test = " + "{:.6f}".format(loss2)
            print "Iter " + str(k) + " ---- Process: " + "{:.2f}".format(100*float(k)/float(num_epoches)) + "%"
        #print "haha{}".format(outp)


Iter 0 ---- Process: 0.00%
Iter 1 ---- Process: 0.07%
Iter 2 ---- Process: 0.13%
Iter 3 ---- Process: 0.20%
Iter 4 ---- Process: 0.27%
Iter 5 ---- Process: 0.33%
Iter 6 ---- Process: 0.40%
Iter 7 ---- Process: 0.47%
Iter 8 ---- Process: 0.53%
Iter 9 ---- Process: 0.60%
Iter 10 ---- Process: 0.67%
Iter 11 ---- Process: 0.73%
Iter 12 ---- Process: 0.80%
Iter 13 ---- Process: 0.87%
Iter 14 ---- Process: 0.93%
Iter 15 ---- Process: 1.00%
Iter 16 ---- Process: 1.07%
Iter 17 ---- Process: 1.13%
Iter 18 ---- Process: 1.20%
Iter 19 ---- Process: 1.27%
Iter 20 ---- Process: 1.33%
Iter 21 ---- Process: 1.40%
Iter 22 ---- Process: 1.47%
Iter 23 ---- Process: 1.53%
Iter 24 ---- Process: 1.60%
Iter 25 ---- Process: 1.67%
Iter 26 ---- Process: 1.73%
Iter 27 ---- Process: 1.80%
Iter 28 ---- Process: 1.87%
Iter 29 ---- Process: 1.93%
Iter 30 ---- Process: 2.00%
Iter 31 ---- Process: 2.07%
Iter 32 ---- Process: 2.13%
Iter 33 ---- Process: 2.20%
Iter 34 ---- Process: 2.27%
Iter 35 ---- Process: 2.33%
It

In [11]:
for i in range(Rs):
    for j in range(cus_num):
        out = np.array(outlist[i])
        tmp = out.T.reshape((1,test_batch_size))
        R[i,j] = np.corrcoef(tmp[0,test_minibatch_size*j:test_minibatch_size*(j+1)],test_y.T[0,test_minibatch_size*j:test_minibatch_size*(j+1)])[0,1]
print R
R1 = np.mean(R,axis=0)
print R1
print np.mean(R1)

[[ 0.65153104  0.71126548  0.53525246  0.56377137  0.58223352  0.97938181
   0.85591773  0.5557183   0.74264783  0.72021615]
 [ 0.65451474  0.71571506  0.51117474  0.56958049  0.57183628  0.97815009
   0.82987763  0.56186522  0.74493124  0.72631591]
 [ 0.65843326  0.71857183  0.52235691  0.57190534  0.57529809  0.97814278
   0.86281149  0.56149128  0.74652264  0.72126982]
 [ 0.66793564  0.72886396  0.51055115  0.56950768  0.58634957  0.97812722
   0.86792258  0.57019509  0.74743371  0.72521861]
 [ 0.65435044  0.71450681  0.49417676  0.56478318  0.57066109  0.97920743
   0.80365358  0.55978989  0.7453884   0.72067559]
 [ 0.66116588  0.72463508  0.5218476   0.57609323  0.57971973  0.9789081
   0.85723334  0.5682576   0.74602695  0.71673205]
 [ 0.66353578  0.7290081   0.52139903  0.56897082  0.58309389  0.97922904
   0.86483145  0.56762353  0.74739209  0.72498749]
 [ 0.66297016  0.72988497  0.51200336  0.57067366  0.5781562   0.97880429
   0.86765213  0.56771217  0.74730004  0.72336569]
 

In [12]:
time2 = time.time()
print time2-time1

4263.76164889
